In [37]:
import data
importlib.reload(data)
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

In [38]:
import variables
import contraintes
import importlib
importlib.reload(variables)
importlib.reload(contraintes)

<module 'contraintes' from 'c:\\Users\\remgo\\Desktop\\ProjetST7\\git\\contraintes.py'>

In [39]:
def show_solution():
    print(var_dict)
    for machine in var_dict.keys():
        print(machine)
        for sillon in var_dict[machine].keys():
            print(var_dict[machine][sillon].X)


In [40]:
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)




m.setObjective(0, GRB.MINIMIZE)

m.update()
m.printStats()
m.presolve()
m.printStats()
m.optimize()

# print(m.objVal)
show_solution()

Number of anti-parallel constraints :  0

Statistics for modelUnnamed:
  Linear constraint matrix    : 47332 Constrs, 24746 Vars, 141996 NZs
  Variable types              : 0 Continuous,
24746 Integer (23666 Binary)
  General constraints         : 23666 Constrs
  Matrix coefficient range    : [ 1, 1e+06 ]
  Objective coefficient range : [ 0, 0 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 15, 999985 ]
Presolve removed 4218 rows and 868 columns (presolve time = 10s) ...
Presolve added 19448 rows and 19986 columns
Presolve time: 9.93s

Statistics for modelUnnamed:
  Linear constraint matrix    : 47332 Constrs, 24746 Vars, 141996 NZs
  Variable types              : 0 Continuous,
24746 Integer (23666 Binary)
  General constraints         : 23666 Constrs
  Matrix coefficient range    : [ 1, 1e+06 ]
  Objective coefficient range : [ 0, 0 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 15, 999985 ]
Gurobi Optimizer version 10.0.0 bui